In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [3]:
# importing the dataset
from keras.datasets import cifar10

# loading in the data     
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d958be074577803d12ecdefd02955f39262c83c16fe9348329d7fe0b5c001ce so we will re-download the data.
170500096/170498071 [==============================] - 1240s 7us/step


In [4]:
# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

class_num = y_test.shape[1]

In [9]:
# creating the model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
#model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(class_num))
model.add(Activation('softmax'))

In [12]:
epochs = 25
optimizer = 'adam'

# compiling the model

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 64)       

In [13]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64) 

Epoch 1/25
782/782 [==============================] - 418s 527ms/step - loss: 1.7456 - accuracy: 0.3963 - val_loss: 1.9886 - val_accuracy: 0.3918
Epoch 2/25
782/782 [==============================] - 396s 506ms/step - loss: 1.0563 - accuracy: 0.6229 - val_loss: 1.1298 - val_accuracy: 0.6076
Epoch 3/25
782/782 [==============================] - 396s 507ms/step - loss: 0.8789 - accuracy: 0.6899 - val_loss: 0.8920 - val_accuracy: 0.6844
Epoch 4/25
782/782 [==============================] - 393s 503ms/step - loss: 0.8043 - accuracy: 0.7196 - val_loss: 0.9309 - val_accuracy: 0.6771
Epoch 5/25
782/782 [==============================] - 401s 513ms/step - loss: 0.7677 - accuracy: 0.7313 - val_loss: 0.8714 - val_accuracy: 0.6908
Epoch 6/25
782/782 [==============================] - 414s 529ms/step - loss: 0.7188 - accuracy: 0.7455 - val_loss: 0.7622 - val_accuracy: 0.7334
Epoch 7/25
782/782 [==============================] - 397s 508ms/step - loss: 0.6886 - accuracy: 0.7574 - val_loss: 0.8183 -

In [14]:
## Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 74.25%
